In [1]:
import pandas as pd
dataset = pd.read_csv('./dataset/dataset_final.csv', header=0, low_memory=False)

In [2]:
dataset.head()

,EDAD,DIST_HECHO,COMISARIA,DIST_CIA,MODALIDAD
0,32,453,371,252,2
1,40,130,311,78,2
2,28,651,431,376,2
3,55,772,598,450,6
4,21,847,664,496,4


In [3]:
dataset['MODALIDAD'].value_counts()

6    177852
5    155965
4     64114
3      6900
2      1453
0       104
1        43
Name: MODALIDAD, dtype: int64

In [4]:
# Vizualizando la diferencia de cantidad de registros por modalidad se decidió no realizar balanceo

In [5]:
X = dataset.drop(['MODALIDAD'], axis=1)
y = dataset['MODALIDAD']

### Separar los datos en entrenamiento y prueba

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### Escalamiento

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Declaración de métricas

In [8]:
def get_metrics(arr):
    tp = arr[1][1]
    tn = arr[0][0]
    fp = arr[0][1]
    fn = arr[1][0]
    return tp, tn, fp, fn

In [9]:
def get_precision(arr):
    tp, tn, fp, fn = get_metrics(arr)
    return tp / (tp + fp)

def get_recall(arr):
    tp, tn, fp, fn = get_metrics(arr)
    return tp / (tp + fn)

def get_f1(arr):
    precision = get_precision(arr)
    recall = get_recall(arr)
    return 2 * ((precision * recall) / (precision + recall))

### Resumen del modelo

In [10]:
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, classification_report
def model_summary(model, metrics):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    ct = pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)
    mcm = multilabel_confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)
    print('Confusion matrix:\n', cm, '\n', ct)
    print('Multilabel confusion matrix:\n', mcm)
    print('Classification report:\n', cr)
    
    for label in range(7):
        metrics.append([get_precision(mcm[label]), get_recall(mcm[label]), get_f1(mcm[label])])

### MODELO 1: Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion='entropy', random_state=0)
dt_m = []
model_summary(dt, dt_m)

dt_mf = pd.DataFrame(dt_m, columns=['Precision', 'Recall', 'F1'])
dt_mf

Confusion matrix:
 [[    1     0     0     0     2     6    18]
 [    0     1     0     0     1     3     3]
 [    0     0    19     1    78   168   183]
 [    0     0     4    27   273   764  1047]
 [    2     0    28    79  4762  6365  8076]
 [    1     1    84   198  4202 24100 18218]
 [    2     1    97   298  5653 19872 27292]] 
 Predicted  0  1    2    3      4      5      6     All
True                                                  
0          1  0    0    0      2      6     18      27
1          0  1    0    0      1      3      3       8
2          0  0   19    1     78    168    183     449
3          0  0    4   27    273    764   1047    2115
4          2  0   28   79   4762   6365   8076   19312
5          1  1   84  198   4202  24100  18218   46804
6          2  1   97  298   5653  19872  27292   53215
All        6  3  232  603  14971  51278  54837  121930
Multilabel confusion matrix:
 [[[121898      5]
  [    26      1]]

 [[121920      2]
  [     7      1]]

 [[1212

,Precision,Recall,F1
0,0.166667,0.037037,0.060606
1,0.333333,0.125000,0.181818
2,0.081897,0.042316,0.055800
3,0.044776,0.012766,0.019868
4,0.318082,0.246582,0.277805
5,0.469987,0.514913,0.491426
6,0.497693,0.512863,0.505164


### MODELO 2: Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
rf_m = []
model_summary(rf, rf_m)

rf_mf = pd.DataFrame(rf_m, columns=['Precision', 'Recall', 'F1'])
rf_mf

Confusion matrix:
 [[    0     0     0     0     2     6    19]
 [    0     0     0     0     1     3     4]
 [    0     0    16     1    57   148   227]
 [    0     0     1    12   220   717  1165]
 [    1     0    17    24  4033  6118  9119]
 [    3     0    45    69  3065 23131 20491]
 [    0     0    42   127  4281 18474 30291]] 
 Predicted  0    2    3      4      5      6     All
True                                               
0          0    0    0      2      6     19      27
1          0    0    0      1      3      4       8
2          0   16    1     57    148    227     449
3          0    1   12    220    717   1165    2115
4          1   17   24   4033   6118   9119   19312
5          3   45   69   3065  23131  20491   46804
6          0   42  127   4281  18474  30291   53215
All        4  121  233  11659  48597  61316  121930
Multilabel confusion matrix:
 [[[121899      4]
  [    27      0]]

 [[121922      0]
  [     8      0]]

 [[121376    105]
  [   433     16]]


c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\renzo\AppData\Local\Temp\ipykernel_13108\10089

,Precision,Recall,F1
0,0.000000,0.000000,NaN
1,NaN,0.000000,NaN
2,0.132231,0.035635,0.056140
3,0.051502,0.005674,0.010221
4,0.345913,0.208834,0.260437
5,0.475976,0.494210,0.484922
6,0.494015,0.569219,0.528957


### MODELO 3: Multilayer Perceptron

In [13]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=0, max_iter=1000)
mlp_m = []
model_summary(mlp, mlp_m)

mlp_mf = pd.DataFrame(mlp_m, columns=['Precision', 'Recall', 'F1'])
mlp_mf

Confusion matrix:
 [[    0     0     0     0     0     5    22]
 [    0     0     0     0     0     3     5]
 [    0     0     0     0     7   127   315]
 [    0     0     0     0    56   681  1378]
 [    0     0     0     0  1048  6796 11468]
 [    0     0     0     0   650 19987 26167]
 [    0     0     0     0   846 14762 37607]] 
 Predicted     4      5      6     All
True                                 
0             0      5     22      27
1             0      3      5       8
2             7    127    315     449
3            56    681   1378    2115
4          1048   6796  11468   19312
5           650  19987  26167   46804
6           846  14762  37607   53215
All        2607  42361  76962  121930
Multilabel confusion matrix:
 [[[121903      0]
  [    27      0]]

 [[121922      0]
  [     8      0]]

 [[121481      0]
  [   449      0]]

 [[119815      0]
  [  2115      0]]

 [[101059   1559]
  [ 18264   1048]]

 [[ 52752  22374]
  [ 26817  19987]]

 [[ 29360  39355]
  [ 156

c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\renzo\AppData\Local\Temp\ipykernel_13108\10089

,Precision,Recall,F1
0,NaN,0.000000,NaN
1,NaN,0.000000,NaN
2,NaN,0.000000,NaN
3,NaN,0.000000,NaN
4,0.401995,0.054267,0.095625
5,0.471825,0.427036,0.448315
6,0.488644,0.706699,0.577783


### MODELO 4: KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7)
knn_m = []
model_summary(knn, knn_m)

knn_mf = pd.DataFrame(knn_m, columns=['Precision', 'Recall', 'F1'])
knn_mf

Confusion matrix:
 [[    0     0     0     0     2     8    17]
 [    0     0     0     0     1     5     2]
 [    0     0     3     0    66   187   193]
 [    0     0     0    11   292   809  1003]
 [    0     0     8    33  4541  6727  8003]
 [    0     0    10    81  4452 23803 18458]
 [    0     0    10    92  5812 20699 26602]] 
 Predicted   2    3      4      5      6     All
True                                           
0           0    0      2      8     17      27
1           0    0      1      5      2       8
2           3    0     66    187    193     449
3           0   11    292    809   1003    2115
4           8   33   4541   6727   8003   19312
5          10   81   4452  23803  18458   46804
6          10   92   5812  20699  26602   53215
All        31  217  15166  52238  54278  121930
Multilabel confusion matrix:
 [[[121903      0]
  [    27      0]]

 [[121922      0]
  [     8      0]]

 [[121453     28]
  [   446      3]]

 [[119609    206]
  [  2104     11]]

 

c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\renzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\renzo\AppData\Local\Temp\ipykernel_13108\10089

,Precision,Recall,F1
0,NaN,0.000000,NaN
1,NaN,0.000000,NaN
2,0.096774,0.006682,0.012500
3,0.050691,0.005201,0.009434
4,0.299420,0.235139,0.263414
5,0.455664,0.508568,0.480665
6,0.490106,0.499897,0.494953
